In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotnine as p9
from functools import partial
import itertools

from learning_to_price_assets.consumption_based_model import (
    transform_wellbeing_power_form, 
    transform_wellbeing_power_form_dwdc, 
    infer_price, 
    transform_lifetime_wellbeing
    )

2025-03-15 13:17:40.040 | INFO     | learning_to_price_assets.config:<module>:11 - PROJ_ROOT path is: /Users/alexabraham/Projects/learning_to_price_assets


In [2]:
# create the grid of pairs (consumption today, tomorrow)
amounts_consumption = np.linspace(0, 1000, 100).round(0).astype(int)

def expand_grid(data_dict):
    """
    SOURCE: https://pandas.pydata.org/docs/user_guide/cookbook.html
    """
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

outcomes_wellbeing = expand_grid({'consumption_today': amounts_consumption, 'consumption_tomorrow': amounts_consumption})

# when wellbeing rate-of-change is the focus, that's undefined for consumption=0
outcomes_wellbeing = outcomes_wellbeing.query("consumption_today > 0 & consumption_tomorrow > 0")

In [3]:
# classical discount factor equals 1 / risk_free_gross_rate; 
# suppose 5% risk free net rate
SUBJECTIVE_DISCOUNT_FACTOR = 1 / 1.05
STRENGTH_OF_DIMINISHING = 0.5

wellbeing_curve = partial(transform_wellbeing_power_form, strength_of_diminishing=STRENGTH_OF_DIMINISHING)

lifetime_wellbeing_curve = partial(
    transform_lifetime_wellbeing, 
    wellbeing_func=wellbeing_curve, 
    subjective_discount_factor=SUBJECTIVE_DISCOUNT_FACTOR
    )

outcomes_wellbeing['lifetime_wellbeing'] = lifetime_wellbeing_curve(
    consumption_today=outcomes_wellbeing['consumption_today'], 
    consumption_expected_tomorrow=outcomes_wellbeing['consumption_tomorrow']
    )

wellbeing_dwdc_curve = partial(transform_wellbeing_power_form_dwdc, strength_of_diminishing=STRENGTH_OF_DIMINISHING)

outcomes_wellbeing = (
    outcomes_wellbeing
    .assign(
        dwdc_tomorrow = lambda df_: wellbeing_dwdc_curve(df_['consumption_tomorrow']),
        dwdc_today = lambda df_: wellbeing_dwdc_curve(df_['consumption_today'])
    )
    .assign(
        dwdc_tomorrow = lambda df_: df_['dwdc_tomorrow'].replace([np.inf, -np.inf], np.nan),
        dwdc_today = lambda df_: df_['dwdc_today'].replace([np.inf, -np.inf], np.nan)
    )
    .assign(
        stochastic_discount_factor = lambda df_: SUBJECTIVE_DISCOUNT_FACTOR * df_['dwdc_tomorrow'] / df_['dwdc_today']
    )
)

In [4]:
# 3d area plotly requires a particular data structure.
# examples reference: https://plotly.com/python/3d-surface-plots/
todays_multi_dimensions = outcomes_wellbeing.pivot(index='consumption_today', columns='consumption_tomorrow')
todays_wellbeing = todays_multi_dimensions['lifetime_wellbeing']
todays_stochastic_discount_factor = todays_multi_dimensions['stochastic_discount_factor']

In [5]:
# PROBLEM: attempts to add color (discount factor) dimension to 3d surface plot,
# and present it in tooltip, yield a consistent error:
# a color value plots with swapped (y, x) coordinate, versus expected (x, y).
# after several iterations, realized this is a known bug: https://github.com/plotly/plotly.js/issues/5003.
# below code follows select recommended practice for (x, y, z, color) dimensions:
#   (https://community.plotly.com/t/surface3d-with-customdata/42708/9)
# additional reference: # https://community.plotly.com/t/customize-hover-text-in-go-surface-plot/45056/5

ranges_consumption_today = todays_wellbeing.index.values
ranges_consumption_tomorrow = todays_wellbeing.columns.values
# meshgrid delivers grid expansion: 
# return1: a matrix of values, from repeating one row vector downwards
# return2: a matrix of values, from repeating one column vector rightwards
ranges_consumption_today, ranges_consumption_tomorrow = np.meshgrid(ranges_consumption_today, ranges_consumption_tomorrow)

grid_lifetime_wellbeing = lifetime_wellbeing_curve(
    consumption_today=ranges_consumption_today, 
    consumption_expected_tomorrow=ranges_consumption_tomorrow
    ).round(1)

grid_stochastic_discount_factor = (
    SUBJECTIVE_DISCOUNT_FACTOR * 
    wellbeing_dwdc_curve(consumption=ranges_consumption_tomorrow) / wellbeing_dwdc_curve(consumption=ranges_consumption_today)
    ).round(2)
grid_stochastic_discount_factor[np.isinf(grid_stochastic_discount_factor)] = np.nan

# recommended workaround for known plotly bug which swaps customdata axes: 
# https://github.com/plotly/plotly.js/issues/5003
coordinates = np.stack(
    (ranges_consumption_today.T, ranges_consumption_tomorrow.T, grid_stochastic_discount_factor.T), 
    axis=-1
    )

In [7]:
titles_axes = dict(
    yaxis_title_text="Consumption Today",
    xaxis_title_text='Consumption <br>Expected Tomorrow', 
    zaxis_title_text="Lifetime Wellbeing"
    )

fig = go.Figure(
    data=[
        go.Surface(
            # axes ordered for easier user interaction
            x=ranges_consumption_tomorrow, 
            y=ranges_consumption_today, 
            z=grid_lifetime_wellbeing, 
            surfacecolor=grid_stochastic_discount_factor, 
            colorbar=dict(title='Random Discount Factor'),
            customdata=coordinates,
            hovertemplate=(
                "Consumption Today: %{y}<br>" + 
                "Consumption Expected Tomorrow: %{x}<br>" + 
                "Random Discount Factor: %{customdata[2]}<br>" + 
                "Lifetime Wellbeing: %{z}<br>" + 
                "<extra></extra>" # remove trace name
                )
            )
        ]
    )

fig.update_traces(
    contours_z=dict(show=True, usecolormap=True, highlightcolor="limegreen", project_z=True),
    )

# better view in the web browser when autosize allowed
fig.update_layout(
    title=dict(text='Discount Factor Rises for Payoffs Arriving in Lean Times'), 
    scene_camera_eye=dict(x=2.5, y=0.88, z=0.5)
    )

# 3d plots axis titles have special access method, vs 2d
# reference: https://community.plotly.com/t/how-to-update-x-y-and-z-axes-titles-for-3d-surface-plots-within-a-subplot/45495
fig.update_scenes(**titles_axes)

fig.show()

fig.write_html("./docs/discount_factor_varies_with_consumption.html", include_plotlyjs='cdn')